In [ ]:
# the initial state and goal state feeder
# TC 1
# initial_state = [["Y", "X"], ["W"], ["Z"]]
# goal_state = [["Z","X"], ["Y", "W"]]

# TC 2
# initial_state = [["Y", "X"], ["Z"], ["W"]]
# goal_state = [["X", "Z"], ["Y", "W"]]

# TC 3
# initial_state = [["Y", "X"], ["Z"], ["W"]]
# goal_state = [["X","Z"], ["W", "Y"]]


# TC 4
initial_state = [["A"], ["E", "C"], ["D", "B"]]
goal_state = [["E"], ["A", "C"], ["D", "B"]]

# TC 5
# initial_state = [["A", "B", "C", "D", "E"]]
# goal_state = [["C", "A", "D"], ["B"], ["E"]]

# TC 6
# initial_state = [["A", "B", "C", "D"]]
# goal_state = [['D', 'C', 'A', 'B']]

# TC 7
# initial_state = [["C", "A"],["B"]]
# goal_state = [["A", "B", "C"]]

# the required sets 
# we use sets as they only take unique values, it helps in streamlining of the code
blocks_on_table = set()
block_on_block = set()
clear_blocks = set()
machine_hand = None

# Takes values from initial state and assigns them to the above mentioned sets
for stack in initial_state:
    clear_blocks.add(stack[0])
    blocks_on_table.add(stack[-1])
    for block in range(len(stack)-1):
        block_on_block.add((stack[block],stack[block+1]))

# another sub-division for printing 
def print_state(state):
    max = 0
    for i in state:
        if len(i) > max:
            max = len(i)
    temp = [[' '] * (max - len(x)) + x for x in state]
    for j in range(max):
        for i in range(len(temp)):
            print(temp[i][j], end = ' ')
        print()
    print('¯ ' * len(temp))

# this function is used to print the initial state
def print_initial_state():
    global initial_state
    print("Initial state")
    print_state(initial_state)

# this function is used to print the goal state
def print_goal_state():
    global goal_state
    print("Goal state")
    print_state(goal_state)

# this function was used by us for debugging, to print and check the values held by each set
def print_globals():
    print(f'Clear = {clear_blocks}')
    print(f'Block on Block = {block_on_block}')
    print(f'On Table = {blocks_on_table}')
    print(f'Machine arm = {machine_hand}')

# this function is used to print the updated states after each operation
def print_state2():
    global blocks_on_table, block_on_block, clear_blocks, machine_hand
    state = ''
    for x in block_on_block:
        state += f"ON({x[0]},{x[1]}) ∧ "
    for x in blocks_on_table:
        state += f"ONTABLE({x}) ∧ "
    for x in clear_blocks:
        state += f"CLEAR({x}) ∧ "
    if machine_hand == None:
        state += "ARMEMPTY"
    else:
        state += f"HOLDING({machine_hand})"
    print(state)
    print()

# this function takes the provided block and removes any block above it 
def unstack(block):
    global clear_blocks, block_on_block, blocks_on_table, machine_hand
    a = block
    b = None
    for i in block_on_block:
        if a==i[1]:
            b = i[0]
            break
    if b==None:
        return
    clear(b)
    on((b,a))
    armEmpty()
    machine_hand = b
    clear_blocks.add(b)
    clear_blocks.add(a)
    block_on_block.remove((b,a))
    print("UnStack", b, a)
    print_state2()

# this function checks if the given block has no block above it
# if yes, then it returns back
# else, it makes sure that no block is present above the required block using the unstack function
def clear(block):
    global clear_blocks, block_on_block, blocks_on_table, machine_hand
    if block in clear_blocks:
        return
    else:
        unstack(block)


# this function checks if the given block is present on the table 
# if yes, then it returns back
# else, it makes sure that the required block is present on the table
def onTable(block):
    global clear_blocks, block_on_block, blocks_on_table, machine_hand
    if block in blocks_on_table:
        return
    else:
      unstack(block)

# this function checks if the arm is empty
# if yes, then it returns back
# else, it makes sure that the arm holds no block
def armEmpty():
    global machine_hand
    if machine_hand==None:
        return
    else:
        print("PutDown", machine_hand)
        blocks_on_table.add(machine_hand)
        machine_hand = None
        print_state2()

# this function checks if the given block is being held in the arm or not
# if yes, then it returns back
# else, it makes sure that the arm hold the required block
def holding(block):
    global machine_hand
    if machine_hand == block:
        return
    else:
        clear(block)
        onTable(block)
        armEmpty()

        machine_hand = block
        print("PickUp", machine_hand)
        print_state2()

# this function take a block-stack and checks if that stack is present in block_on_block array
# if yes, then it returns back
# else, it makes sure that the condition is fullfilled
def on(blocks):
    global machine_hand, block_on_block, clear_blocks
    if blocks in block_on_block :
        return
    else:
        clear(blocks[1])
        holding(blocks[0])
        print("Stack", blocks[0], blocks[1])

        clear_blocks.remove(blocks[1])
        clear_blocks.add(blocks[0])
        block_on_block.add(blocks)
        blocks_on_table.remove(blocks[0])
        machine_hand = None
        print_state2()

# the program starts here

# first prints the initial state and the goal state of the given problem
print_initial_state()
print_goal_state()
# then it prints the initial state of program in predicate variables
print_state2()

for i in range(2):
  for stack in goal_state:                      # the stack holds the columns from the goal state 
      clear(stack[0])                           # first we clear the top most element in the stack to match it with the goal state
      for block in range(len(stack)-2, -1, -1):
          on((stack[block], stack[block+1]))    # then we check if the requiredblocks are present on top of one another
      onTable(stack[-1])                        
      armEmpty()


Initial state
  E D 
A C B 
¯ ¯ ¯ 
Goal state
  A D 
E C B 
¯ ¯ ¯ 
ON(E,C) ∧ ON(D,B) ∧ ONTABLE(B) ∧ ONTABLE(C) ∧ ONTABLE(A) ∧ CLEAR(E) ∧ CLEAR(A) ∧ CLEAR(D) ∧ ARMEMPTY

UnStack E C
ON(D,B) ∧ ONTABLE(B) ∧ ONTABLE(C) ∧ ONTABLE(A) ∧ CLEAR(E) ∧ CLEAR(C) ∧ CLEAR(A) ∧ CLEAR(D) ∧ HOLDING(E)

PutDown E
ON(D,B) ∧ ONTABLE(B) ∧ ONTABLE(E) ∧ ONTABLE(C) ∧ ONTABLE(A) ∧ CLEAR(E) ∧ CLEAR(C) ∧ CLEAR(A) ∧ CLEAR(D) ∧ ARMEMPTY

PickUp A
ON(D,B) ∧ ONTABLE(B) ∧ ONTABLE(E) ∧ ONTABLE(C) ∧ ONTABLE(A) ∧ CLEAR(E) ∧ CLEAR(C) ∧ CLEAR(A) ∧ CLEAR(D) ∧ HOLDING(A)

Stack A C
ON(D,B) ∧ ON(A,C) ∧ ONTABLE(B) ∧ ONTABLE(E) ∧ ONTABLE(C) ∧ CLEAR(E) ∧ CLEAR(A) ∧ CLEAR(D) ∧ ARMEMPTY

